In [16]:
# Python==3.8 (3.7.10 backup plan)

!pip install spacy
!pip install transformers
!pip install fastcoref    # Used for coreference resolution
!pip install intervaltree # Used for finding overlapping intervals between sentence start and end positions and mention positions within the text. 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)


Attempting a fetch of a provided web page

In [4]:
import trafilatura
downloaded = trafilatura.fetch_url('https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/')
trafilatura.extract(downloaded)

'Leveraging AI to empower all developers at GitHub Universe 2023\nGitHub Universe 2023 is just around the corner! Join us and learn how GitHub is leveraging AI to empower all developers, including developers with disabilities.\nWe’re spending Women’s History Month with women leaders who are making history every day in the tech community. Read more about Erin Spiceland: Software Engineer at SpaceX.\nEvery March we recognize the women who have shaped history—and now, we’re taking a look forward. From driving software development in large companies to maintaining thriving open source communities, we’re spending Women’s History Month with women leaders who are making history every day in the tech community. Erin Spiceland is a Software Engineer for SpaceX. Born and raised in rural south Georgia, she is a Choctaw and Chickasaw mother of two now living in downtown Los Angeles. Erin didn’t finish college—she’s a predominantly self-taught software engineer. In her spare time, she makes handmad

Attempting a fetch of URLs from an RSS (Really Simple Syndication) feed 

In [17]:
from trafilatura import feeds
mylist = feeds.find_feed_urls('https://www.dailymail.co.uk/news/rishi-sunak/index.rss') # mylist = feeds.find_feed_urls('https://www.dailymail.co.uk/news/uk-politics/index.rss')
print(mylist)

['https://www.dailymail.co.uk/?ito=1490&ns_campaign=1490&ns_mchannel=rss', 'https://www.dailymail.co.uk/debate/article-12663637/alex-brummer-joe-biden-bond-markets-sunak-starmer.html?ito=1490&ns_campaign=1490&ns_mchannel=rss', 'https://www.dailymail.co.uk/debate/article-12663701/NADINE-DORRIES-Googled-learnt-Big-Tech.html?ito=1490&ns_campaign=1490&ns_mchannel=rss', 'https://www.dailymail.co.uk/debate/article-12663827/QUENTIN-LETTS-Jeremy-Corbyn-Rishi-Sunak-crisply-recalled-hed-hailed-Hamas-friends.html?ito=1490&ns_campaign=1490&ns_mchannel=rss', 'https://www.dailymail.co.uk/debate/article-12672813/STEPHEN-GLOVER-rocket-Jeremy-Hunt-doesnt-promise-tax-cuts-Tories-finished.html?ito=1490&ns_campaign=1490&ns_mchannel=rss', 'https://www.dailymail.co.uk/debate/article-12677181/QUENTIN-LETTS-Sunaks-gaze-framed-begging-dog-eyebrows-told-AI-herald-mankinds-obliteration.html?ito=1490&ns_campaign=1490&ns_mchannel=rss', 'https://www.dailymail.co.uk/debate/article-12678233/Banning-China-UKs-AI-summi

https://adrien.barbaresi.eu/blog/how-to-download-parallel-politeness-rules-python.html
Provides a method to check the robot txt of a site and check if the url being extracted meets the politeness policy.

When extracting from bing check for each domain and store in data structure with allow/disallow. If the same base url appears again then no need to recheck robot.txt

Consider when implementing News Article fetch adding politeness / adherence to robot policy.

In [18]:
import urllib.robotparser

# define a website to look for rules
base_url = "https://www.theguardian.com"

# load the necessary components, fetch and parse the file
rules = urllib.robotparser.RobotFileParser()
rules.set_url(base_url + "/robots.txt")
rules.read()

print(rules)

# determine if a page can be fetched by all crawlers
rules.can_fetch("*", "https://www.theguardian.com/import urllib.robotparser")
# returns True or False

User-agent: Mediapartners-Google
Allow: 

User-agent: NewsNow
Disallow: /

User-agent: GPTBot
Disallow: /

User-agent: CCBot
Disallow: /

User-agent: TurnitinBot
Disallow: /

User-agent: PetalBot
Disallow: /

User-agent: MoodleBot
Disallow: /

User-agent: *
Disallow: /sendarticle/
Disallow: /Users/
Disallow: /users/
Disallow: /%2A/print%24
Disallow: /email/
Disallow: /contactus/
Disallow: /share/
Disallow: /websearch
Disallow: /%2A%3Fcommentpage%3D
Disallow: /whsmiths/
Disallow: /external/overture/
Disallow: /discussion/report-abuse/%2A
Disallow: /discussion/report-abuse-ajax/%2A
Disallow: /discussion/comment-permalink/%2A
Disallow: /discussion/report-abuse/%2A
Disallow: /discussion/user-report-abuse/%2A
Disallow: /discussion/handlers/%2A
Disallow: /discussion/your-profile
Disallow: /discussion/your-comments
Disallow: /discussion/edit-profile
Disallow: /discussion/search/comments
Disallow: /discussion/%2A
Disallow: /search
Disallow: /music/artist/%2A
Disallow: /music/album/%2A
Disallow

True

Extraction from 1 article from the RSS list and setting some extra parameters in Trafilatura.

In [19]:
downloaded = trafilatura.fetch_url(mylist[6])
trafilatura.extract(downloaded, favour_precision=True, include_comments=False, include_images=False, include_tables=False)

"Banning China from the UK's AI summit would make no sense, writes former head of GCHQ's national cyber security centre CIARAN MARTIN\nThe Prime Minister trod a difficult line in yesterday's landmark speech on Artificial Intelligence.\nOn the one hand, he was right to downplay fears that the technology might bring about the apocalypse. Rishi Sunak understands that, despite the hysterical warnings of some Silicon Valley gurus, we shouldn't fear killer robots enslaving or extinguishing our species.\nInstead, as he pointed out, AI is revolutionary – and that revolution brings opportunities, especially in medical research and making our economy more productive.\nBut it undoubtedly brings risks, too. And as head of cyber security at GCHQ for seven years, I learned to appreciate them.\nAs Sunak outlined yesterday, AI could help rogue states unleash new cyber-attacks with unprecedented penetration, design devastating biochemical weapons and generate vast amounts of disinformation that hostile

In [ ]:
'''Article is saved as text file anyway'''
# downloaded = trafilatura.fetch_url('https://www.dailymail.co.uk/news/article-12350195/Think-twice-imposing-Ulez-tax-hard-working-Brits-Rishi-Sunak-blasts-Sadiq-Khan-hated-low-emission-zone-expansion-plan.html?ns_mchannel=rss&ns_campaign=1490&ito=1490')
# article_text = trafilatura.extract(downloaded)

Saving article to file - useful for demo/consistent experimental purposes - if article is deleted or edited my copy will stay the same.

In [ ]:
# file_path = "ULEZ-daily-mail-research-article.txt"
# 
# # Write the text to the file
# with open(file_path, 'w', encoding='utf-8') as file:
#     file.write(article_text)
# 
# print(f"Article text saved to {file_path}")

In [20]:
file_path = "ULEZ-daily-mail-research-article.txt"

# Read the text from the file
with open(file_path, 'r', encoding='utf-8') as file:
    article_text = file.read()

# Now, article_text contains the text you read from the file
print(f"Article text read from {file_path}:\n{article_text}")    

Article text read from ULEZ-daily-mail-research-article.txt:
'Think twice before imposing Ulez tax on hard-working Brits': Rishi Sunak blasts Sadiq Khan for his hated low emission zone expansion plan
- Keir Starmer told to 'get off the fence' over Sadiq Khan's Ulez by senior Tory MPs
- Rishi Sunak led MPs asking Starmer to get London mayor to postpone the levy
- READ MORE: Labour civil war over ULEZ intensifies after Khan's High Court win
Keir Starmer faced fresh calls last night for him to try to delay the expansion of London's clean air zone.
Senior Tories said he should 'get off the fence' and force the capital's mayor Sadiq Khan to protect motorists set to pay the Ulez levy.
The High Court yesterday backed the extension of Ulez into outer London – a move that could lead to similar schemes elsewhere in Britain.
Sir Keir has refused to say whether the zone should expand, claiming it is not a 'simple political decision'. He remained silent after yesterday's court ruling.
Rishi Sunak l

In [21]:
import torch
import spacy

In [22]:
# Check Metal Performance Shaders (MPS) backend for GPU training acceleration.
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [24]:
from fastcoref import FCoref

model = FCoref(device='mps')

preds = model.predict(
   texts=['We are so happy to see you using our coref package. This package is very fast!']
)

preds[0].get_clusters(as_strings=False)


print(preds[0].get_clusters())


preds[0].get_logit(
   span_i=(33, 50), span_j=(52, 64)
)


11/06/2023 23:41:25 - INFO - 	 missing_keys: []
11/06/2023 23:41:25 - INFO - 	 unexpected_keys: []
11/06/2023 23:41:25 - INFO - 	 mismatched_keys: []
11/06/2023 23:41:25 - INFO - 	 error_msgs: []
11/06/2023 23:41:25 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
11/06/2023 23:41:25 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

11/06/2023 23:41:26 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[['We', 'our'], ['our coref package', 'This package']]


18.852913

Playing around with extracting coreference clusters using fastcoref directly
"Processing can be applied to a collection of texts of any length in a batched and parallel fashion"
https://pypi.org/project/fastcoref/

In [12]:
'''
I am performing coreference resolution using fastcoref on a given text and then analysing and printing information about the resolved coreference clusters. 
This helps me identify and understand how words or phrases in the text refer to the same entities,
and I am prioritising clusters based on their text content length.

Sorted_combined_clusters will be used later with the NER resulted from spaCy.
'''



from fastcoref import FCoref

model = FCoref(device='mps')

preds = model.predict(
   texts=[article_text]
)

clustersText = preds[0].get_clusters()
clustersPositions = preds[0].get_clusters(as_strings=False)

print(clustersText)

combined_clusters = [(text, positions, len(text)) for text, positions in zip(clustersText, clustersPositions)]

sorted_combined_clusters = sorted(combined_clusters, key=lambda x: x[2], reverse=True)

for i, (clusterText, clusterPos, cluster_text_count) in enumerate(sorted_combined_clusters, 1):
    print(f"Cluster {i} : {clusterText}")
    print(f"Cluster Text Count: {cluster_text_count}")
    print("Positions:", clusterPos)


11/06/2023 23:36:38 - INFO - 	 missing_keys: []
11/06/2023 23:36:38 - INFO - 	 unexpected_keys: []
11/06/2023 23:36:38 - INFO - 	 mismatched_keys: []
11/06/2023 23:36:38 - INFO - 	 error_msgs: []
11/06/2023 23:36:38 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
11/06/2023 23:36:38 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

11/06/2023 23:36:38 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[['Sadiq Khan', 'his', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'I', 'I', 'Mr Khan', 'Sadiq', 'He', 'he', 'his', 'he', 'your mayor', 'the Labour mayor', 'his', "Mr Khan's", "the mayor's", 'his', 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"], ['Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak (pictured)'], ['Keir Starmer', 'Starmer', 'Keir Starmer', 'him', 'he', 'Sir Keir', 'He', 'the Labour leader', 'Keir Starmer (left)', 'him', 'the Labour leader', "Sir Keir Starmer's", 'Starmer', 'Sir Keir', 'the Labour leader', 'Keir', 'he', 'Sir Keir', 'Sir Keir'], ["Sadiq Khan's Ulez", 'ULEZ', 'ULEZ'], ['London', "London's", "the capital's", 'London', "London's", 'London', 'London', "London's", 'London', "London's", 'London'], ['the levy', 'the Ulez levy', 'the £12.50 daily levy for older, more polluting vehicles', 'it', 'the £12.50 dai

LingMess Coref - alternative state-of-the-art coreference resolution but doesn't want to play with my M1 Mac GPU so have to fall back to slower CPU device :(

In [26]:
from fastcoref import LingMessCoref

model = LingMessCoref(device='cpu')

preds = model.predict(
   texts=[article_text]
)

print(preds[0].get_clusters()) 

11/06/2023 23:42:30 - INFO - 	 missing_keys: []
11/06/2023 23:42:30 - INFO - 	 unexpected_keys: []
11/06/2023 23:42:30 - INFO - 	 mismatched_keys: []
11/06/2023 23:42:30 - INFO - 	 error_msgs: []
11/06/2023 23:42:30 - INFO - 	 Model Parameters: 590.0M, Transformer: 434.6M, Coref head: 155.4M
11/06/2023 23:42:30 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

11/06/2023 23:42:30 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[['Sadiq Khan', 'his', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'I', 'I', 'Sir Keir', 'the Labour leader', 'Mr Khan', 'Keir', 'he', 'Sadiq', 'He', 'he', 'his', 'he', 'Sir Keir', 'your mayor', 'Sir Keir', 'the Labour mayor', 'his', "Mr Khan's", "the mayor's", 'his', 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"], ['Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak (pictured)'], ['Keir Starmer', 'Starmer', 'Keir Starmer', 'him', 'he', 'Sir Keir', 'He', 'the Labour leader', 'Keir Starmer (left)', 'him', 'the Labour leader', "Sir Keir Starmer's", 'Starmer'], ['Ulez', 'ULEZ', 'Ulez', 'Ulez', 'Ulez', 'ULEZ', 'Ulez', 'the Ulez', 'Ulez', 'Ulez', 'the policy', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'the scheme', 'Ulez'], ['London', "London's", "the capital's", 'London', 'London', "London's", 'London', 'London', "London


But to my current knowledge: The spaCy library itself does not natively support processing an array of text documents in a single call in parallel.
So consider applying fastCoref directly then using spaCy or alternative to extract sentences

https://medium.com/quantrium-tech/top-3-packages-for-named-entity-recognition-e9e14f6f0a2a
    
And so decided to play around with flair

Spacy NER recommended so applied here:

Possible NER outcomes: CARDINAL, DATE, EVENT, FAC, GPE, LANGUAGE, LAW, LOC, MONEY, NORP, ORDINAL, ORG, PERCENT, PERSON, PRODUCT, QUANTITY, TIME, WORK_OF_ART
https://huggingface.co/spacy/en_core_web_sm

NER could be confused and additional mentions could be used to determine "correct" category

In [31]:
'''The insert_intervals function enables the segmentation of sentences provided by TextBlob (as shown in the cell below) to be further divided into smaller boundaries. This division is based on the identification of specific points where it is deemed necessary to split sentences, particularly within the context of news articles.'''

def insert_intervals(initial_list, new_values):
    def insert_recursive(intervals, values):
        if not values:
            return intervals # Base case: Return the intervals when there are no more values to insert.

        value = values[0]
        result = []
        for interval in intervals:
            if interval[0] <= value <= interval[1]:
                # If the value falls within an existing interval, split the interval into two parts.
                # The first part goes from the interval's start to the value (inclusive), and
                # the second part goes from the value+1 to the interval's end.
                if interval[0] < value:
                    result.append((interval[0], value))         # To mess around with intervals change value + - offset here.
                if value < interval[1]:
                    result.append((value+1, interval[1]))       # To mess around with intervals change value + - offset here.
            else:
                # If the value doesn't fall within the interval, keep the interval as is.
                result.append(interval)
        return insert_recursive(result, values[1:]) # Recursively process other values.

    updated_list = insert_recursive(initial_list, new_values) # Recursive function call
    return updated_list

In [32]:
'''Process the article text, tokenize by sentence and add custom adjustments to the tokenization using insert intervals above'''

from textblob import TextBlob
import re

# spaCy was not satisfactory for accurately tokenizing for sentence start / end characters. Trying textblob instead...
def textblob_custom_sentences(article_text):
    # Process the article text and adjust tokenization
    blob = TextBlob(article_text)
    sentences = blob.sentences

    # Determine custom tokenization:
    hyphen_sentences = re.split(r'\n-', article_text)
    hyphen_nl_pos = [pos for pos, char in enumerate(article_text) if article_text[pos:pos+2] == '\n-']  # In testing article a new line and '-' hyphen use typically means consider a different sentence.
    extra_split = hyphen_nl_pos

    sentence_bounds = [(int(sentence.start), int(sentence.end)) for sentence in sentences]

    # Insert custom intervals
    updated_list = insert_intervals(sentence_bounds, extra_split)

    return updated_list

In [33]:
'''Merge all instances of the same entity into a single entry with multiple positions. Where same means a lowercase word match. Place them into an 'entities' dictionary.'''

def merge_positions(entities, word):
    entity_key = (word.text + word.label_).lower()
    if entity_key in entities:
        entities[entity_key][1].append([word.start_char, word.end_char])
    else:
        entities[entity_key] = [word.text, [[word.start_char, word.end_char]], word.label_]
    return entities

[(0, 138), (139, 223), (224, 301), (302, 489), (490, 626), (627, 764), (765, 874), (875, 925), (926, 1108), (1109, 1139), (1140, 1605), (1606, 1963), (1964, 2098), (2099, 2298), (2299, 2468), (2469, 2768), (2769, 2937), (2938, 3074), (3075, 3220), (3221, 3273), (3274, 3447), (3448, 3548), (3549, 3710), (3711, 3736), (3737, 3770), (3771, 3820), (3821, 3887), (3888, 3931), (3932, 4009), (4010, 4198), (4199, 4396), (4397, 4607), (4608, 4805), (4806, 4981), (4982, 5077), (5078, 5163), (5164, 5240), (5241, 5371), (5372, 5415), (5416, 5621), (5622, 5738), (5739, 5954), (5955, 6166), (6167, 6254), (6255, 6391), (6392, 6553), (6554, 6613), (6614, 6770), (6771, 6956), (6957, 7268), (7269, 7444), (7445, 7604), (7605, 7691), (7692, 7735), (7736, 7856), (7857, 7909)]
56
Coreference Threshold: 3
Entity Type: CARDINAL
Entity: nearly 700,000
Positions: [[2131, 2145]]
Label: CARDINAL
Number of Positions: 1

Entity: More than 1.5million
Positions: [[2299, 2319]]
Label: CARDINAL
Number of Positions: 1



In [34]:
import spacy
import math
from functools import reduce

NER = spacy.load("en_core_web_sm")

article_text = [article_text][0]
article = NER(article_text)

# Store sentence positions for use with coreference mentions and news sentiment later
sentence_bounds = textblob_custom_sentences(article_text)
num_sentences = len(list(sentence_bounds))
print(sentence_bounds)
print(num_sentences)

# NER sentence results comparison
# for sent in article.sents:
#     print(sent.start_char)
#     print(sent.end_char)
# 
# num_sentences = len(list(article.sents))
# print(num_sentences)

'''# Recommended mention - 'Discard a cluster c in a document d if |Mc| ≤ 0.2|Sd|, where |...| is the 
number of mentions of a cluster (Mc) and sentences in a document (Sd) (NEWS-MTSC approach)'''

# MENTION_REQ_PER = 0.20 

MENTION_REQ_PER = 0.07 # Keep as this for ULEZ research article findings later on.
threshold = math.floor(num_sentences * MENTION_REQ_PER)
print(f"Coreference Threshold: {threshold}")

# num_sentences = len(list(article.sents))
# sentence_positions = [(sent.start_char, sent.end_char) for sent in article.sents]

# # Print sentences + start and end pos.
# for i, (start, end) in enumerate(sentence_positions):
#     sentence_text = article[start:end].text
#     print(f"Sentence {i+1}: {sentence_text}")
#     print(f"Start Position: {start}, End Position: {end}")
#     print()

entity_types = ["CARDINAL", "DATE", "EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOC", "MONEY", "NORP", "ORDINAL", "ORG", "PERCENT", "PERSON", "PRODUCT", "QUANTITY", "TIME", "WORK_OF_ART"]

entity_type_to_entities = {
    entity_type: [
        [
            entity_text,
            positions,
            label,
            len(positions)  # Count of positions
        ] for entity_text, positions, label in reduce(
            merge_positions,
            filter(lambda word: word.label_ == entity_type, article.ents),
            {}
        ).values()
    ] for entity_type in entity_types
}

for entity_type, entities in entity_type_to_entities.items():
    print(f"Entity Type: {entity_type}")
    for entity in entities:
        entity_text, positions, label, num_positions = entity
        print(f"Entity: {entity_text}")
        print(f"Positions: {positions}")
        print(f"Label: {label}")
        print(f"Number of Positions: {num_positions}")
        print()
        
people_entities = {entity_type: entity_info for entity_type, entity_info in entity_type_to_entities.items() if entity_type == 'PERSON'}

for entity in entity_type_to_entities[entity_type]:
    entity_text, positions, label, num_positions = entity
    if entity_type not in people_entities:
        people_entities[entity_type] = []
    people_entities[entity_type].append({
        'Entity': entity_text,
        'Positions': positions,
        'Label': label,
        'Number of Positions': num_positions
    })

Entity Type Contained in Dict: PERSON
Entity: Rishi
Positions: [[62, 67], [1269, 1274]]
Label: PERSON
Number of Positions: 2

Entity: Sadiq Khan
Positions: [[81, 91], [569, 579], [1243, 1253], [1735, 1745]]
Label: PERSON
Number of Positions: 4

Entity: Keir Starmer
Positions: [[141, 153], [382, 394], [1140, 1152], [1483, 1495]]
Label: PERSON
Number of Positions: 4

Entity: Tory
Positions: [[215, 219]]
Label: PERSON
Number of Positions: 1

Entity: Khan
Positions: [[360, 364], [1009, 1013], [1363, 1367], [2102, 2106], [2506, 2510], [3396, 3400], [5918, 5922], [6345, 6349], [7106, 7110]]
Label: PERSON
Number of Positions: 9

Entity: Keir
Positions: [[769, 773], [3079, 3083], [3613, 3617], [4090, 4094], [5283, 5287]]
Label: PERSON
Number of Positions: 5

Entity: Rishi Sunak
Positions: [[926, 937]]
Label: PERSON
Number of Positions: 1

Entity: Starmer
Positions: [[1688, 1695]]
Label: PERSON
Number of Positions: 1

Entity: Boris Johnson's
Positions: [[3034, 3049]]
Label: PERSON
Number of Pos

In [35]:
for entity_type, entities in people_entities.items():
    print(f"Entity Type Contained in Dict: {entity_type}")
    for entity in entities:
        entity_text = entity[0]
        positions = entity[1]
        label = entity[2]
        num_positions = entity[3]
        print(f"Entity: {entity_text}")
        print(f"Positions: {positions}")
        print(f"Label: {label}")
        print(f"Number of Positions: {num_positions}")
        print()


19
{'PERSON': [['Rishi', [[62, 67], [1269, 1274]], 'PERSON', 2], ['Sadiq Khan', [[81, 91], [569, 579], [1243, 1253], [1735, 1745]], 'PERSON', 4], ['Keir Starmer', [[141, 153], [382, 394], [1140, 1152], [1483, 1495]], 'PERSON', 4], ['Tory', [[215, 219]], 'PERSON', 1], ['Khan', [[360, 364], [1009, 1013], [1363, 1367], [2102, 2106], [2506, 2510], [3396, 3400], [5918, 5922], [6345, 6349], [7106, 7110]], 'PERSON', 9], ['Keir', [[769, 773], [3079, 3083], [3613, 3617], [4090, 4094], [5283, 5287]], 'PERSON', 5], ['Rishi Sunak', [[926, 937]], 'PERSON', 1], ['Starmer', [[1688, 1695]], 'PERSON', 1], ["Boris Johnson's", [[3034, 3049]], 'PERSON', 1], ['Rachel Reeves', [[3466, 3479], [3696, 3709]], 'PERSON', 2], ['Wes Streeting', [[3579, 3592]], 'PERSON', 1], ['Grant Shapps', [[4027, 4039]], 'PERSON', 1], ['Ulez', [[4200, 4204]], 'PERSON', 1], ['Trader', [[4811, 4817]], 'PERSON', 1], ['Ian Plummer', [[4996, 5007]], 'PERSON', 1], ['Lucy Frazer', [[5259, 5270]], 'PERSON', 1], ['Mark Harper', [[5436, 5

In [ ]:
print(sorted_combined_clusters[1][2])
print(people_entities)

In [ ]:


print(f"Number of Sentences: {num_sentences}")
print("Coreference threshold: {:.2f} ({}% detected cluster mentions versus sentences)".format(threshold, int(MENTION_REQ_PER * 100)))
print("Cluster Entities before filtration: ", len(sorted_combined_clusters))

filtered_clusters = [(i, cluster) for i, cluster in enumerate(sorted_combined_clusters) if cluster[2] >= threshold]

print("Cluster Entities after filtration: ", len(filtered_clusters))

print(filtered_clusters)

In [ ]:
'''
First draft - too simplistic see below markdown cell.
'''
# entity_to_cluster_mapping = {}
# 
# for entity_type, entities in people_entities.items():
#     for entity in entities:
#         entity_name = entity[0]
#         for cluster in filtered_clusters:
#             cluster_text = cluster[0]
#             for coref_entry in cluster_text:
#                 if entity_name in coref_entry:
#                     if entity_name not in entity_to_cluster_mapping:
#                         entity_to_cluster_mapping[entity_name] = []
# 
#                     entity_to_cluster_mapping[entity_name].append((cluster_text, cluster[1]))
#                     break
# print(entity_to_cluster_mapping) 
# Replaced by beneath code 

Entity to cluster mapping initially was is this NER entity containined in any coreference cluster.
This was too simplistic for example:

{'Entity Name': 'Keir Starmer', 'Positions': [[141, 153], [382, 394], [1140, 1152], [1483, 1495]], 'Label': 'PERSON', 'Num Positions': 4, 'Cluster Info': [{'Cluster Text': ['Sadiq Khan', 'his', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'I', 'I', 'Mr Khan', 'Sadiq', 'He', 'he', 'his', 'he', 'your mayor', 'the Labour mayor', 'his', "Mr Khan's", "the mayor's", 'his', 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"]. 
- Yes 'Keir' is in the coreference mention 'Sir Keir Starmer's mayor' but on balance wouldn't you agree these coreferences are not referring to him?


Solutions I have thought about: 
1. Removing pronouns (his, her, he, I etc) then doing a % match rate on entity text across the cluster. In the example above we would expect Sadiq / Khan to win out.

2. Removing pronouns (his, her, he, I etc) then doing a % match rate on entity position values across the cluster positions.

In [ ]:
entity_to_cluster_mapping = []

# For matching NER people extractions with coreference these references are not useful.
undesired_words = ["i", "he", "his", "she", "they", "it", "this", "that", "these", "those", "the", "a", "an", "of"]


def cleanse_cluster_text(cluster_text):
    return [word.strip() for word in cluster_text if word.lower().strip() not in undesired_words]

threshold_percentage = 0.15 # If 

for entity_type, entities in people_entities.items():
    for entity in entities:
        entity_name, positions, label, num_positions = entity
        entity_entry = {
            'Entity Name': entity_name,
            'Positions': positions,
            'Label': label,
            'Num Positions': num_positions,
            'Cluster Info': []
        }

        for index, (cluster_text, cluster_positions, _) in filtered_clusters:

            # for coref_entry in cluster_text:
            #     if entity_name in coref_entry:
            #         entity_entry['Cluster Info'] = cluster_entry
            #         break
           
            cleaned_cluster_text = cleanse_cluster_text(cluster_text)
            total_coref_words = " ".join(cleaned_cluster_text)
            entity_count = total_coref_words.count(entity_name)
            percentage = entity_count / len(cleaned_cluster_text)
            if percentage > 0.00:
                print(entity_name)
                print(cleaned_cluster_text)
                print(percentage)
                print('------------------------------------------')
        
            if percentage >= threshold_percentage:
                cluster_entry = {                
                'Cluster Text': cluster_text,
                'Cluster Positions': cluster_positions
            }
                
                entity_entry['Cluster Info'] = cluster_entry
                break

        entity_to_cluster_mapping.append(entity_entry)


Evaluation for threshold:

Rishi
['Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak (pictured)']
1.0 Correct
------------------------------------------
Sadiq Khan
['Sadiq Khan', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'Mr Khan', 'Sadiq', 'your mayor', 'the Labour mayor', "Mr Khan's", "the mayor's", 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"]
0.22727272727272727 Correct
------------------------------------------
Sadiq Khan
["the expansion of London's clean air zone", "the expansion of London mayor Sadiq Khan's (right) Ulez", 'expansion', 'the Ulez extension', 'the policy', 'expand', 'the Ulez expansion', 'his Ulez expansion', 'expand', 'expand', "the expansion of London's clean air zone", 'the change', 'Ulez expansion']
0.07692307692307693 Not correct
------------------------------------------
Sadiq Khan
["London's clean air zone", 'Ulez', 'the zone', 'The zone', "London mayor Sadiq Khan's (right) Ulez", 'the expanded Ulez zone', 'the Ulez', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'the scheme', "London's clean air zone", 'Ulez']
0.0625 Not correct
------------------------------------------
Keir Starmer
['Sadiq Khan', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'Mr Khan', 'Sadiq', 'your mayor', 'the Labour mayor', "Mr Khan's", "the mayor's", 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"]
0.045454545454545456 Not correct
------------------------------------------
Keir Starmer
['Keir Starmer', 'Starmer', 'Keir Starmer', 'him', 'Sir Keir', 'the Labour leader', 'Keir Starmer (left)', 'him', 'the Labour leader', "Sir Keir Starmer's", 'Starmer', 'Sir Keir', 'the Labour leader', 'Keir', 'Sir Keir', 'Sir Keir']
0.25 Correct
------------------------------------------
Khan
['Sadiq Khan', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'Mr Khan', 'Sadiq', 'your mayor', 'the Labour mayor', "Mr Khan's", "the mayor's", 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"]
0.6363636363636364 Correct
------------------------------------------
Keir
['Sadiq Khan', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'Mr Khan', 'Sadiq', 'your mayor', 'the Labour mayor', "Mr Khan's", "the mayor's", 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"]
0.045454545454545456 Not correct
------------------------------------------
Keir
['Keir Starmer', 'Starmer', 'Keir Starmer', 'him', 'Sir Keir', 'the Labour leader', 'Keir Starmer (left)', 'him', 'the Labour leader', "Sir Keir Starmer's", 'Starmer', 'Sir Keir', 'the Labour leader', 'Keir', 'Sir Keir', 'Sir Keir']
0.5625 Correct
------------------------------------------
Rishi Sunak
['Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak (pictured)']
1.0 Correct
------------------------------------------
Starmer
['Sadiq Khan', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'Mr Khan', 'Sadiq', 'your mayor', 'the Labour mayor', "Mr Khan's", "the mayor's", 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"]
0.045454545454545456 Not correct
------------------------------------------
Starmer
['Keir Starmer', 'Starmer', 'Keir Starmer', 'him', 'Sir Keir', 'the Labour leader', 'Keir Starmer (left)', 'him', 'the Labour leader', "Sir Keir Starmer's", 'Starmer', 'Sir Keir', 'the Labour leader', 'Keir', 'Sir Keir', 'Sir Keir']
0.375 Correct
------------------------------------------
Rachel Reeves
['Shadow Chancellor Rachel Reeves', 'Rachel Reeves', 'Mr Justice Swift']
0.6666666666666666 Correct
------------------------------------------
Ulez
["the expansion of London's clean air zone", "the expansion of London mayor Sadiq Khan's (right) Ulez", 'expansion', 'the Ulez extension', 'the policy', 'expand', 'the Ulez expansion', 'his Ulez expansion', 'expand', 'expand', "the expansion of London's clean air zone", 'the change', 'Ulez expansion']
0.38461538461538464 Correct
------------------------------------------
Ulez
["London's clean air zone", 'Ulez', 'the zone', 'The zone', "London mayor Sadiq Khan's (right) Ulez", 'the expanded Ulez zone', 'the Ulez', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'Ulez', 'the scheme', "London's clean air zone", 'Ulez']
0.6875 Correct
------------------------------------------
On this whole entity name approach with the above data I figure 0.15. Let's see if we can get a higher threshold value somehow... 

Let us try spliting the entity names into part words e.g. Sadiq Khan will be split into Sadiq and Khan for evaluation purposes.

In [ ]:
entity_to_cluster_mapping = []

undesired_words = ["i", "he", "his", "she", "they", "it", "this", "that", "these", "those", "the", "a", "an", "of"]


def cleanse_cluster_text(cluster_text):
    return [word.strip() for word in cluster_text if word.lower().strip() not in undesired_words]

threshold_percentage = 0.30

for entity_type, entities in people_entities.items():
    for entity in entities:
        entity_name, positions, label, num_positions = entity
        entity_entry = {
            'Entity Name': entity_name,
            'Positions': positions,
            'Label': label,
            'Num Positions': num_positions,
            'Cluster Info': []
        }

        cluster_id = 0

        for index, (cluster_text, cluster_positions, _) in filtered_clusters:
            
            cluster_id += 1
            
            cleaned_cluster_text = cleanse_cluster_text(cluster_text)
            total_coref_words = " ".join(cleaned_cluster_text)
            
            entity_parts = entity_name.split()
            max_percentage = 0.0

            for entity_part in entity_parts:
                entity_count = total_coref_words.count(entity_part)
                percentage = entity_count / len(cleaned_cluster_text) #  How well the entity name matches the cluster. 
            
                if percentage > max_percentage: # Is this a better match than the previous best match? If so make this the new best match.
                    max_percentage = percentage
                    
                if percentage > 0.00:
                    print("Entity Part: " + entity_part)
                    print(cleaned_cluster_text)
                    print(percentage)
                    print('------------------------------------------')

            if max_percentage > 0.00:
                print("Entity Name: " + entity_name)
                print(cleaned_cluster_text)
                print("Max % Match: " + str(max_percentage))
                if max_percentage > threshold_percentage:
                    print("PREDICTION:  MATCH")
                else:
                    print ("PREDICTION: INVALID")
                print('------------------------------------------')
                   
                    
            cleaned_cluster_text = cleanse_cluster_text(cluster_text)
            total_coref_words = " ".join(cleaned_cluster_text)
            entity_count = total_coref_words.count(entity_name)
            percentage = entity_count / len(cleaned_cluster_text)
            
        
            if percentage >= threshold_percentage:
                cluster_entry = {  
                'Cluster ID' : cluster_id,
                'Cluster Text': cluster_text,
                'Cluster Positions': cluster_positions
            }
                
                entity_entry['Cluster Info'] = cluster_entry
                entity_to_cluster_mapping.append(entity_entry)
                break

        


On this new evaluation will double threshold now to p = 0.3 -- could further explore requirement on the start/end position values in text

In [ ]:
# print(entity_to_cluster_mapping)

# Bin entities that were not successful in coreference combination stage i.e their cluster info is blank.

for entry in entity_to_cluster_mapping:
    print(entry)
    print()

We need to consolidate some entries that have been determined as the same cluster ID.

Two types identified so far:
    1. where an entity is the substring of another e.g. Rishi is a substring of Rishi Sunak so should be merged into the 'better' later version with full name.
    2. where an entity first and second name are separate entity lists but one is the first name e.g., Keir and the other is the second name Starmer. I propose trying a combination of 2 of the entities under the same cluster ID and checking if they match any coreference values in cluster text 
    (validating that first / second name combination) so then we can break out create a new combined entity and remove the first / second name instance.


In [ ]:
'''
Purposes of below code:

1. Substring Matching: If one entity is a substring of another entity, they are considered as candidates for consolidation. For example, if "Rishi" is a substring of "Rishi Sunak," the code merges them into the longer version, "Rishi Sunak."

2. First and Second Name Combination: If there are two entities, one representing the first name and the other representing the last name, and they share the same coreference cluster, the code attempts to combine them into a single entity.'''


from collections import defaultdict

cluster_dict = defaultdict(list)

for entry in entity_to_cluster_mapping:
    entity_name = entry['Entity Name']
    cluster_id = entry['Cluster Info']['Cluster ID']
    
    cluster_dict[cluster_id].append(entry)


def is_substring(entity1, entity2):
    return entity1 in entity2 or entity2 in entity1

def combine_entities(entities, cluster_text):
    combined_entity = None

    for entity1 in entities:
        for entity2 in entities:
            if entity1 != entity2:
                combined1 = entity1 + ' ' + entity2
                combined2 = entity2 + ' ' + entity1

                if combined1 in cluster_text or combined2 in cluster_text:
                    combined_entity = combined1 if combined1 in cluster_text else combined2
                    break

    return combined_entity

for cluster_id, entries in cluster_dict.items():
    if len(entries) > 1:
        combined_entry = None
        for i, entry1 in enumerate(entries):
            for j, entry2 in enumerate(entries):
                if i < j:
                    entity_1_name = entry1['Entity Name']
                    entity_2_name = entry2['Entity Name']
                    
                    cluster_text = entries[0]['Cluster Info']['Cluster Text']
                    combined_entity = combine_entities([entity_1_name, entity_2_name], cluster_text)
                    
                    
                    if is_substring(entity_1_name, entity_2_name):
                        if len(entity_1_name) > len(entity_2_name):
                            entries.remove(entry2)
                            entry1['Num Positions'] = int(-200) # Coreference cluster will be used anyway -200 to indicate merge via removal.
                            entry1['Positions'] = int(-200) 
                        else:
                            entries.remove(entry1)
                            entry2['Num Positions'] = int(-200) # Coreference cluster will be used anyway -200 to indicate merge via removal.
                            entry2['Positions'] = int(-200) 
                            
                    elif combined_entity in cluster_text:
                        combined_entry = {
                            'Entity Name': combined_entity,
                            'Positions': int(-100), # Coreference cluster will be used anyway -100 to indicate merge via combined entity.
                            'Label': entry1['Label'], 
                            'Num Positions': int(-100),  # Coreference cluster will be used anyway -100 to indicate merge via combined entity.
                            'Cluster Info': entry1['Cluster Info'] 
                        }
                        entries.remove(entry1)
                        entries.remove(entry2)
                        break
        if combined_entry:
            entries.append(combined_entry)                

clustered_entities = [entry for entries in cluster_dict.values() for entry in entries]

for entry in clustered_entities:
    print(entry)
    print()


With Khan may be worth searching the cluster text for two word mentions that ft. Khan to extract Sadiq Khan as a "better" entity name.

In [ ]:
# !pip install textblob

In [ ]:
# ***Replaced with TextBlob code below as it had a convenient sentence.start and sentence.end)


# import nltk
# from nltk.tokenize import sent_tokenize
# 
# sentences = sent_tokenize(article_text)
# sentence_boundaries = [(0, len(sentences[0]))] 
# for i in range(1, len(sentences)):
#     start = sentence_boundaries[-1][1] + 1
#     end = start + len(sentences[i])
#     sentence_boundaries.append((start, end))
# 
# 
# # print(sentence_boundaries)
# # Cluster positions (assuming they are character positions within the text)
# cluster_positions = [(20, 35)]
# 
# # Initialise a list to store the selected sentences
# selected_sentences = []
# 
# # Map cluster positions to sentences
# for start, end in cluster_positions:
#     for i, (sentence_start, sentence_end) in enumerate(sentence_boundaries):
#         if sentence_start <= start < sentence_end or sentence_start <= end < sentence_end:
#             selected_sentences.append(sentences[i])
# 
# # Print the selected sentences
# for sentence in selected_sentences:
#     print(sentence)

In [ ]:
# Old Boundaries Code:


# def insert_intervals(initial_list, new_values):
#     def insert_recursive(intervals, values):
#         if not values:
#             return intervals
# 
#         value = values[0]
#         result = []
#         for interval in intervals:
#             if interval[0] <= value <= interval[1]:
#                 if interval[0] < value:
#                     result.append((interval[0], value - 1))
#                 if value < interval[1]:
#                     result.append((value, interval[1]))
#             else:
#                 result.append(interval)
#         return insert_recursive(result, values[1:])
# 
#     updated_list = insert_recursive(initial_list, new_values)
#     return updated_list

In [ ]:
print(sentence_bounds)
print(extra_split)

In [ ]:
'''The purpose of this graph is to illustrate the impact of different exponents on the scaling of sentiment probabilities.'''


import matplotlib.pyplot as plt
import numpy as np

# Probabilities from 0 to 1
probabilities = np.linspace(0, 1, 100)

exponents = range(11)

# Create subplots for each exponent
plt.figure(figsize=(12, 8))

for exponent in exponents:
    points = (probabilities ** exponent) * 100
    plt.plot(probabilities, points, label=f'Exponent = {exponent}')

plt.xlabel('Probability')
plt.ylabel('Points')
plt.title('Exponential Scaling of Sentiment Probabilities')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
def scaling_old(probabilities, k=3, linear=False):
    neutral_points = positive_points = negative_points = 0

    for prob_data in probabilities:
        class_prob = prob_data['class_prob']
        class_label = prob_data['class_label']
        
        if linear:
            points = class_prob * 100
        else:
            points = (class_prob ** k) * 100

        if class_label == 'neutral':
            neutral_points += points
        elif class_label == 'positive':
            positive_points += points
        elif class_label == 'negative':
            negative_points += points

    return [neutral_points, positive_points, negative_points]

In [ ]:
def scaling(avg_array_list, k=3, linear=False):
    neutral_points = positive_points = negative_points = 0

    for avg_array in avg_array_list:
        for i, avg_value in enumerate(avg_array):
            
            if linear:
                points = avg_value * 100
            else:
                points = (avg_value ** k) * 100
    
            if i == 0:
                neutral_points += points
            elif i == 1:
                positive_points += points
            elif i == 2:
                negative_points += points

    return [neutral_points, positive_points, negative_points]

In [ ]:
def average_array(probabilities):
    num_probabilities = len(probabilities)
    neutral_total = positive_total = negative_total = 0

    for prob_data in probabilities:
        class_prob = prob_data['class_prob']
        class_label = prob_data['class_label']

        if class_label == 'neutral':
            neutral_total += class_prob
        elif class_label == 'positive':
            positive_total += class_prob
        elif class_label == 'negative':
            negative_total += class_prob

    neutral_avg = neutral_total / num_probabilities if num_probabilities > 0 else 0
    positive_avg = positive_total / num_probabilities if num_probabilities > 0 else 0
    negative_avg = negative_total / num_probabilities if num_probabilities > 0 else 0

    return [neutral_avg, positive_avg, negative_avg]

In [ ]:
# !pip install NewsSentiment

from NewsSentiment import TargetSentimentClassifier
tsc = TargetSentimentClassifier()

def bounds_sentiment(mention_start, mention_end, sentence_start, sentence_end):
    
    sentiment = tsc.infer_from_text(article_text[sentence_start:mention_start] , article_text[mention_start:mention_end], article_text[mention_end:sentence_end])
    print(sentiment[0])
    return sentiment[0]
    
    

In [ ]:
# Code to identify sentence and highlight target - sending first instance of entity in a sentence to NewsSentiment. 
# Misses some sentences that are un-punctuated in the ULEZ research article.

from intervaltree import Interval, IntervalTree

print(clustered_entities[1]['Entity Name'])

START_HIGHLIGHT = '\033[0m'
END_HIGHLIGHT = '\033[94m'

GREEN = '\033[92m'
END_COLOR = '\033[0m'

cluster_positions = [(81, 91), (96, 99), (187, 199), (268, 280), (360, 366), (549, 579), (1006, 1013), (1230, 1255), (1360, 1367), (1479, 1503), (1735, 1745), (2099, 2108), (2503, 2510), (2669, 2670), (2688, 2689), (3393, 3400), (3741, 3746), (3771, 3773), (3796, 3798), (3866, 3869), (3936, 3938), (4136, 4146), (5326, 5342), (5352, 5355), (5915, 5924), (5999, 6010), (6154, 6157), (6342, 6349), (7103, 7110), (7216, 7225), (7468, 7479)]



bounds_tree = IntervalTree(Interval(start, end) for start, end in sentence_bounds)

processed_bounds = {}

for mention_start, mention_end in cluster_positions:
    overlap = bounds_tree.overlap(mention_start, mention_end)
    if overlap:
        first_unprocessed = None
        for interval in overlap:
            print(" ")
            sentence_start, sentence_end = interval.begin, interval.end
            bounds_key = (sentence_start, sentence_end)
            
            highlighted_text = (
                START_HIGHLIGHT + article_text[sentence_start:mention_start] + END_HIGHLIGHT +
                article_text[mention_start:mention_end] +
                START_HIGHLIGHT + article_text[mention_end:sentence_end] + END_HIGHLIGHT)
            print(START_HIGHLIGHT + f"Mention ({mention_start}, {mention_end}) is within bounds ({sentence_start}, {sentence_end})")
            print(highlighted_text)
            
            
            if bounds_key not in processed_bounds:
                first_unprocessed = interval
                processed_bounds[bounds_key] = True
                bounds_sentiment(mention_start, mention_end, sentence_start, sentence_end)
                print(GREEN + "NewsSentiment Candidate as first bound appearance" + END_COLOR)
                break

In [ ]:
bounds_data = {}

for entity in clustered_entities:
    entity_name = entity['Entity Name']
    cluster_positions = entity['Cluster Info']['Cluster Positions']


    for mention_start, mention_end in cluster_positions:
        overlap = bounds_tree.overlap(mention_start, mention_end)
        if overlap:
            for interval in overlap:
                print(" ")
                sentence_start, sentence_end = interval.begin, interval.end
                bounds_key = (sentence_start, sentence_end)

                if bounds_key not in bounds_data:
                    bounds_data[bounds_key] = {}

                if entity_name not in bounds_data[bounds_key]:
                    bounds_data[bounds_key][entity_name] = []

                highlighted_text = (
                    START_HIGHLIGHT + article_text[sentence_start:mention_start] + END_HIGHLIGHT +
                    article_text[mention_start:mention_end] +
                    START_HIGHLIGHT + article_text[mention_end:sentence_end] + END_HIGHLIGHT)
                print(START_HIGHLIGHT + f"{entity_name} - Mention ({mention_start}, {mention_end}) is within bounds ({sentence_start}, {sentence_end})")
                print(highlighted_text)

                result = bounds_sentiment(mention_start, mention_end, sentence_start, sentence_end)

                bounds_data[bounds_key][entity_name].append(result)

                print(GREEN + f"NewsSentiment Candidate appearance {len(bounds_data[bounds_key][entity_name])}" + END_COLOR)


In [ ]:
def percentage_contribution(elements):
    total = sum(elements)
    percentage_contributions = [(element / total) * 100 for element in elements]
    return percentage_contributions

In [ ]:
entity_averages = {}

for bounds_key, entity_results in bounds_data.items():
    for entity_name, results in entity_results.items():
        print(entity_name + " in:")
        print(article_text[bounds_key[0]:bounds_key[1]])
        print(results)
        avg = average_array(results)
        
        if entity_name not in entity_averages:
            entity_averages[entity_name] = []
        entity_averages[entity_name].append(avg)
    
    
for entity_name, averages in entity_averages.items():
    print(f"Averages for {entity_name}:")
    for avg in averages:
        print(avg)
    scaled_classification =scaling(averages, k=3)
    percentage_values = percentage_contribution(scaled_classification)
    
    linear_scaled_classification =scaling(averages, linear=True)
    linear_percentage_values = percentage_contribution(linear_scaled_classification)
    
    print("Under exponential points system: ")
    print(scaled_classification)
    print(percentage_values)
    
    
    
    if scaled_classification[0] > scaled_classification[1] and scaled_classification[0] > scaled_classification[2]:
        print("Exponential Neutral Majority")
        
    elif scaled_classification[1] > scaled_classification[0] and scaled_classification[1] > scaled_classification[2]:
        print("Exponential Positive Majority")
        
    elif scaled_classification[2] > scaled_classification[0] and scaled_classification[2] > scaled_classification[1]:
        print("Exponential Negative Majority")
        
    print("Under linear points system: ")
    print(linear_scaled_classification)
    print(linear_percentage_values)
    
    if linear_scaled_classification[0] > linear_scaled_classification[1] and linear_scaled_classification[0] > linear_scaled_classification[2]:
        print("Linear Neutral Majority")
        
    elif linear_scaled_classification[1] > linear_scaled_classification[0] and linear_scaled_classification[1] > linear_scaled_classification[2]:
        print("Linear Positive Majority")
        
    elif linear_scaled_classification[2] > linear_scaled_classification[0] and linear_scaled_classification[2] > linear_scaled_classification[1]:
        print("Linear Negative Majority")

    print("----------------")
    


In [ ]:
collated_entities = {}

for entity_type, entities in entity_type_to_entities.items():
    for entity_info in entities:
        entity_text = entity_info[0]
        positions = entity_info[1]
        label = entity_info[2]

        if entity_text in collated_entities:
            if label in collated_entities[entity_text]:
                collated_entities[entity_text][label]['occurrences'] += len(positions)
                collated_entities[entity_text][label]['positions'].extend(positions)
            else:
                collated_entities[entity_text][label] = {'occurrences': len(positions), 'positions': positions}
        else:
            collated_entities[entity_text] = {label: {'occurrences': len(positions), 'positions': positions}}


entities_with_multiple_labels = {}

# for entity_text, label_occurrences in collated_entities.items():
#     output = [f"Entity Text: {entity_text}"]
#     for label, data in label_occurrences.items():
#         output.append(f"Label: {label}, Occurrences: {data['occurrences']}, Positions: {data['positions']}")
#     print(" | ".join(output))
    

# Print results where there are duplicate / in dispute NER labels:

# for entity_text, label_occurrences in collated_entities.items():
#     # Check if there are multiple labels associated with the entity text
#     if len(label_occurrences) > 1:
#         output = [f"Entity Text: {entity_text}"]
#         for label, data in label_occurrences.items():
#              output.append(f"Label: {label}, Occurrences: {data['occurrences']}, Positions: {data['positions']}")
#              # output.append(f"Label: {label}, Occurrences: {data['occurrences']}")
#         entities_with_multiple_labels[entity_text] = " | ".join(output)
#         print(" | ".join(output))
#         
# 
# 
# for entity_text, data in entities_with_multiple_labels.items():
#     print(f" {data}")

# Store results where there are duplicate / in dispute NER labels:
for entity_text, label_occurrences in collated_entities.items():
    if len(label_occurrences) > 1:
        entities_with_multiple_labels[entity_text] = []
        for label, data in label_occurrences.items():
            entity_info = {
                "Label": label,
                "Occurrences": data['occurrences'],
                "Positions": data['positions']
            }
            entities_with_multiple_labels[entity_text].append(entity_info)

# Print the stored alike entity text values that have multiple NER labels:
for entity_text, label_data in entities_with_multiple_labels.items():
    print("---------------------------")
    print(f"Entity Text: {entity_text}")
    for label_info in label_data:
        print(f"Label: {label_info['Label']}, Occurrences: {label_info['Occurrences']}, Positions: {label_info['Positions']}")


Appears using the majority case wouldn't always work. 

Let us park these inconclusive NER classifications here ^^^. After we have connected the NER to the coreference resolution output....

In [ ]:
nlp = spacy.load("en_core_web_sm")

coreference_matrix = [
    'Sadiq Khan', 'his', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan",
    'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan',
    "Mr Khan's", 'Mr Khan', 'I', 'I', 'Mr Khan', 'Sadiq', 'He', 'he', 'his', 'he', 'your mayor',
    'the Labour mayor', 'his', "Mr Khan's", "the mayor's", 'his', 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"
]

split_matrix = [element.split() for element in coreference_matrix]

nouns = []
for words in split_matrix:
    for word in words:
        doc = nlp(word)
        for token in doc:
            if token.pos_ == "NOUN":
                nouns.append(token.text)

print(nouns)

Seems flawed approach as Keir has been brought out from the phrase "Sir Keir Starmer's mayor" even when we are referring to Sadiq Khan - additionally Khan was discarded too.

In [ ]:
for cluster in clustersText:
    print("-----------------------------")
    most_specific_mention = None
   
    text = " ".join(cluster)
    doc = nlp(text)

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            most_specific_mention = most_specific_mention or ent.text

    if most_specific_mention:
        print(cluster)
        print("Most Specific PERSON Mention:", most_specific_mention)
    else:
        print(cluster)
        print("No specific PERSON name found in the cluster.")

In [ ]:
# TODO: Resolve conflict code above to identify prevalence of NER unsure.
# TODO: Match co-reference resolution outcomes with NER identification to increase positions code.
# TODO: WIKIPEDIA Resolution? (Nice to have? for summaries?)
# TODO: Extract sentences for all identified mentions
# TODO: Apply NewsSentiment with correct start/end positions and formatting resolution